In [1]:
import os
import gc
import shutil
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score
from microns_phase3 import nda
from ccmodels.preprocessing.connectomics import client_version, subset_v1l234
from ccmodels.preprocessing.selectivity import orientation_extractor, von_mises, von_mises_single, is_selective, cell_area_identifiers


Connecting microns@db.datajoint.com:3306


Could not create the ~log table
Could not access table `microns_phase3_nda`.`~log`


In [3]:
#Identify desired neurons, in this case v1 neurons from L234
client = client_version(661)
area_v1_neurons = cell_area_identifiers('V1')
v1l234_neur = subset_v1l234(client, table_name = 'coregistration_manual_v3', area_df = area_v1_neurons)
v1l234_neur = v1l234_neur[v1l234_neur['pt_root_id'] != 0]

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


In [15]:
def identify_multiscan(df, id_col = 'pt_root_id', count_col = 'session'):

    rootid_groups = df.groupby([id_col]).count().reset_index()
    multiscan_ids = rootid_groups[rootid_groups[count_col] > 1][id_col].values

    return multiscan_ids

def fpd_assignment(session, scan_idx):
    if session == 9:
        if scan_idx == 6:
            fpd = 9
        else:
            fpd = 8
    else:
        fpd = 6
    return fpd

In [13]:
multiscan_ids = identify_multiscan(v1l234_neur)

In [14]:
v1l234_singlescan = v1l234_neur[~v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_multiscan = v1l234_neur[v1l234_neur['pt_root_id'].isin(multiscan_ids)]
v1l234_singlescan.shape, v1l234_multiscan.shape, v1l234_neur.shape

((5067, 9), (2025, 9), (7092, 9))

In [37]:
#Container with saved data
data = []
            
#Von Mises
columns = ['root_id', 'session', 'scan_idx','cell_id', 'pvalue','tuning_type', 
        'r_squared_diff', 'mean_r_sqrd', 'A', 'phi', 'k', 'activity', 'orientations']

for mult_id in tqdm(multiscan_ids, desc = f'Extracting multiscan neurons data') :
    subset_multiscan = v1l234_multiscan[v1l234_multiscan['pt_root_id'] == mult_id]
    scan_acts = []
    
    for row in range(subset_multiscan.shape[0]):
        unit_key = {'session':subset_multiscan['session'].values[row], 'scan_idx':subset_multiscan['scan_idx'].values[row], 'unit_id':subset_multiscan['unit_id'].values[row]}
        fpd = fpd_assignment(subset_multiscan['session'].values[row], subset_multiscan['scan_idx'].values[row])
        df = orientation_extractor(unit_key, fpd)
        scan_acts.append(df)
    
    all_acts = pd.concat(scan_acts, axis = 1)

    #Calculating max activity for constraining von mises, for plotting data and fit
    gp = df.groupby('radians').mean().reset_index()
    max_act_rad = gp.sort_values('mean_activity', ascending = False)['radians'].values[0]
            
    #Using Von Mises
    pars_d, pcov_d = curve_fit(lambda theta, A, k: von_mises(theta, A, max_act_rad, k), df['radians'], df['mean_activity'],
                                bounds = ([-np.inf, 0.1],[np.inf, np.inf]),
                                method = 'trf')

    pars_s, pcov_s = curve_fit(lambda theta, A, k: von_mises(theta, A, max_act_rad, k), df['radians'], df['mean_activity'],
                                bounds = ([-np.inf, 0.1],[np.inf, np.inf]),
                                method = 'trf')



    # Von Mises activities
    ate_d= [von_mises(i,pars_d[0], max_act_rad,  pars_d[1]) for i in gp['radians']]
    ate_s= [von_mises_single(i,pars_s[0], max_act_rad,  pars_s[1]) for i in gp['radians']]
            
    r2d = r2_score(gp['mean_activity'], ate_d)
    r2s = r2_score(gp['mean_activity'], ate_s)

    #Calculate difference in r2 score between the two model fist to identify fringe cases
    rdiff = r2d-r2s
            
    #single
    vs, ps, max_rad = is_selective(df, max_act_rad)


    #double
    vdb, pdb, max_rad = is_selective(df, max_act_rad, single = False)
            
            
    #save data
    data.append([mult_id, unit_key['session'], unit_key['scan_idx'], unit_key['unit_id'],ps,'single', rdiff, r2s, 
                        pars_d[0], max_act_rad,  pars_d[1], np.array(gp['mean_activity']), np.array(gp['radians'])])
            
    #save data
    data.append([mult_id, unit_key['session'], unit_key['scan_idx'], unit_key['unit_id'],pdb,'double', rdiff,r2d, 
                        pars_s[0], max_act_rad,  pars_s[1], np.array(gp['mean_activity']), np.array(gp['radians'])])
            
#Save the data
data_df = pd.DataFrame(data, columns = columns)
data_df.to_pickle(f'../data/in_processing/orientation_fits/multiscan_fits.pkl')


Extracting multiscan neurons data:   0%|          | 0/967 [00:00<?, ?it/s]

/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


KeyboardInterrupt: 

In [32]:
data_df

,root_id,session,scan_idx,cell_id,pvalue,tuning_type,r_squared_diff,mean_r_sqrd,A,phi,k,activity,orientations
0,864691134885761018,5,7,1647,0.404938,single,-0.171106,0.079112,0.429890,0.785398,0.100000,"[0.43700632, 0.33167058, 0.7850223, 0.17658675...","[0.0, 0.39269908169872414, 0.7853981633974483,..."
1,864691134885761018,5,7,1647,0.627285,double,-0.171106,-0.091994,0.429890,0.785398,0.100000,"[0.43700632, 0.33167058, 0.7850223, 0.17658675...","[0.0, 0.39269908169872414, 0.7853981633974483,..."
2,864691134917966090,4,7,6291,0.301062,single,0.840813,-0.316979,10.587981,4.319690,0.520136,"[3.5795207, 4.148244, 11.047732, 7.842968, 8.0...","[0.0, 0.39269908169872414, 0.7853981633974483,..."
3,864691134917966090,4,7,6291,0.000714,double,0.840813,0.523834,10.587981,4.319690,0.520136,"[3.5795207, 4.148244, 11.047732, 7.842968, 8.0...","[0.0, 0.39269908169872414, 0.7853981633974483,..."


In [18]:
t = pd.read_pickle('/workspaces/microns_phase3_nda/data/in_processing/orientation_fits.pkl')

In [11]:
nda.Scan().fetch(format = 'frame')

nframes  nfields      fps
session scan_idx                           
4       7           40000        8  6.30090
        9           35112        8  6.30090
        10          40000        8  6.30090
5       3           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
6       2           40000        8  6.30090
        4           40000        8  6.30090
        6           40000        8  6.30090
        7           40000        8  6.30090
7       3           40000        8  6.30090
        4           40000        8  6.30090
        5           40000        8  6.30090
8       5           40000        8  6.30090
        7           40000        8  6.30090
        9           40000        8  6.30090
9       3           50000        6  8.61754
        4           50000        6  8.61754
        6           57000        4  9.62246

In [15]:
ors_all = pd.read_pickle(f"../data/in_processing/orientation_fits/{os.listdir('../data/in_processing/orientation_fits')[0]}")

#Loading the rest iteratively and concatenating
for file in tqdm(os.listdir('../data/in_processing/orientation_fits/')[1:], desc='Aggregating session, scan files'):
    if file!='.DS_Store':
        cont_df = pd.read_pickle(f'../data/in_processing/orientation_fits/{file}')
        ors_all = pd.concat([ors_all, cont_df], axis = 0)
    else:
        continue

#Saving the data
ors_all.to_pickle('../data/in_processing/orientation_fits.pkl')

Aggregating session, scan files:   0%|          | 0/15 [00:00<?, ?it/s]

In [16]:
shutil.rmtree('../data/in_processing/orientation_fits')